In [119]:
import numpy as np
import math

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.

In [120]:
def vec_mod(vec):
    S = 0
    for i in range(vec.shape[0]):
        S = S + (vec[i])**2
    return math.sqrt(S)  

def householder(vec):
    
    n = vec.shape[0]
    
    x = vec_mod(vec)
    y = np.zeros(n)
    y[0] = x
    u = (vec-y)/vec_mod(vec-y)
    
    u = np.array(u)[np.newaxis]
    H = np.eye(n) - 2 * np.matrix.transpose(u) @ u
    
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
        
    return y, H

Test your function using tests below:

In [121]:
# Test I.1 (10% of the total grade).

v = np.array([1, 2, 3])
v1, h = householder(v)

np.testing.assert_allclose(np.dot(h, v1), v)
np.testing.assert_allclose(np.dot(h, v), v1)

# <CHECK>
# print(v1)
# print(h @ v)

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

Mismatch: 33.3%
Max absolute difference: 0.
Max relative difference: nan
 x: array([ 3.741657,  0.      , -0.      ])
 y: array([3.741657, 0.      , 0.      ])

In [122]:
# Test I.2 (10% of the total grade).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

np.testing.assert_allclose(np.dot(h, v1), vec)


# <CHECK>
# print(v1)
# print(h @ vec)

# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

Write a function, which receives a recangular matrix, $A$, and returns the Q and R factors of the $QR$ factorization of $A$.

In [123]:
rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))

def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    m = a.shape[0]
    n = a.shape[1]
    q = np.eye(m)
    r = a
    
    for i in range(n):
        
        vec = r[i::, i].copy()   
        y, H0 = householder(vec)  
        H1 = np.eye(m)
        
        for j in range(m-i):
            for k in range(m-i):
                H1[j+i][k+i] = H0[j][k]
        
        r = H1 @ r
        q = q @ np.transpose(H1)
    
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape 
    
    return q, r

In [124]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [125]:
# Test II.1 (20% of the total grade)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [127]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

print('<=INITIAL MATRIX=>')
print('A')
print(a)
print('')
print('<=QR-decomposition given by lybrary=>')
print('Q')
print(qq)
print('R')
print(rr)
print('')
print('<=My QR decomposition=>')
print('Q')
print(q)
print('R')
print(r)
print('')
print('<=Matrices a, qq*rr, q*r=>')
print(a)
print(qq @ rr)
print(q @ r)

<=INITIAL MATRIX=>
A
[[0.19151945 0.62210877 0.43772774]
 [0.78535858 0.77997581 0.27259261]
 [0.27646426 0.80187218 0.95813935]
 [0.87593263 0.35781727 0.50099513]
 [0.68346294 0.71270203 0.37025075]]

<=QR-decomposition given by lybrary=>
Q
[[-0.13665049  0.53601299  0.09369752  0.661619   -0.49749149]
 [-0.56035895  0.0935397   0.53326881 -0.52477245 -0.34276292]
 [-0.19725922  0.65948912 -0.60068463 -0.37879015  0.14784752]
 [-0.62498418 -0.50418303 -0.52144688  0.18967657 -0.21750907]
 [-0.48765568  0.12171264  0.27224305  0.32774225  0.75222783]]
R
[[-1.40152769 -1.2514379  -0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.         -0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]

<=My QR decomposition=>
Q
[[ 0.13665049  0.53601299 -0.09369752  0.7697136   0.30459557]
 [ 0.56035895  0.0935397  -0.53326881  0.01839528 -0.62652547]
 [ 0.19725922  0.65948912  0.60068463 -0.32384673 -0.24589462]
 [ 0.62498418 -0.50418303  

Check if your `q` and `r` agree with `qq` and `rr`. Explain.

My explanation: if A - quadratic nondegenerate matrix and diagonal elements of matrix R are positive and real, QR-decomposition is uniqe. In our task A - is not diagonal. Consequently library QR-dec. and my can differ.

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests. 

(Peer-graded, 40% of the total grade)

In [128]:
# <=COMPUTES VECTOR, THAT REFLECTS X=>
def reflection_vector(vec): 
    n = vec.shape[0]
    x = vec_mod(vec)
    y = np.zeros(n)
    y[0] = x
    u = (vec-y)/vec_mod(vec-y)
    
    return u

# <=RECONSTRUCTION OF QT MATRIX FROM REFLECTION VECTORS=>
# (that given in order in list)
def QT_reconstruction(u):
    m = len(u[0])
    n = len(u[-1])
    qt = np.eye(m)
    
    for i in range(n): 
        H = np.eye(m)
        
        # Insert uu^T in unit matrix on every step
        for j in range(m-i):
            for k in range(m-i):
                H[j+i][k+i] = H[j+i][k+i] - 2*u[i][j]*u[i][k]
                
        # Tracking matrices H
        # print(h)
        
        qt = H @ qt
    
    return qt
        
    
def qr_avoiding_decomp(a):
    
    m = a.shape[0]
    n = a.shape[1]
    q = np.eye(m)
    r = a.copy()
    
    u = []
    
    for i in range(n):
        
        vec = r[i::, i].copy()   
        u0 = reflection_vector(vec)      
        u.append(u0)
        u0 = np.array(u0)[np.newaxis]
        vec = np.array(vec)[np.newaxis]
        
        # Doing "cheap Housholder pivot - O(mn)" on every step
        r[i::,i::] = r[i::,i::] - 2 * np.matrix.transpose(u0) @ (u0 @ r[i::,i::])
        
        # Tracking change of matrix
        # print(r)
    
    return u, r

print('<=INITIAL MATRIX=>')
print('A')
print(a)
print('')
print('')

u, r = qr_avoiding_decomp(a)

print('<=DECOMPOSITION AVOIDING EXPLICIT FORMING H=>')
print("<=REFLECTION VECTORS=>")
print(u)
print("R")
print(r)
print('')

qt = QT_reconstruction(u)

print('<=RECONSTRUCTION OF QT FROM REFLECTION VECTORS=>')
print('QT')
print(qt)
print('')
print('<=CHECK THAT QT*A IS FOUND TRIANGULAR MATRIX=>')
print('QT*A')
print(qt @ a)


<=INITIAL MATRIX=>
A
[[0.19151945 0.62210877 0.43772774]
 [0.78535858 0.77997581 0.27259261]
 [0.27646426 0.80187218 0.95813935]
 [0.87593263 0.35781727 0.50099513]
 [0.68346294 0.71270203 0.37025075]]


<=DECOMPOSITION AVOIDING EXPLICIT FORMING H=>
<=REFLECTION VECTORS=>
[array([-0.6570196 ,  0.42644006,  0.15011669,  0.47562065,  0.37111197]), array([-0.52846942,  0.73983285, -0.10990213,  0.40160796]), array([-0.79133207,  0.36468006, -0.49071581])]
R
[[ 1.40152769  1.2514379   0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.          0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.          0.         -0.        ]]

<=RECONSTRUCTION OF QT FROM REFLECTION VECTORS=>
QT
[[ 0.13665049  0.56035895  0.19725922  0.62498418  0.48765568]
 [ 0.53601299  0.0935397   0.65948912 -0.50418303  0.12171264]
 [-0.09369752 -0.53326881  0.60068463  0.52144688 -0.27224305]
 [ 0.7697136   0.01839528 -0.32384673  0.28453698 -0.47049398]
 [ 0.30459557 -0.62652547 -0.24589462  0